In [1]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *

Using TensorFlow backend.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

In [3]:
train_emb, train_emb_api, train_fam_ans, train_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/TRAIN_vec.pkl','rb'))
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/DEV_vec.pkl','rb'))
test_emb, test_emb_api,test_fam_ans,test_rep_ans = pickle.load(open('data/tree-rep-profiles-partial/TEST_vec.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)
emb_matrix = pickle.load(open('data/tree-rep-profiles-partial/api_emb_matrix.pkl','rb'))

test of sent2vec vector: (424, 213, 768) (424, 213) (424, 44) (424, 213, 1)


In [4]:
# train only fam hash unique
def unique_hash(train_emb,train_emb_api,train_fam_ans,train_rep_ans):
    unique , indx = np.unique(train_emb, axis=0, return_index=True)
    emb_api = train_emb_api[indx]
    fam = train_fam_ans[indx]
    rep = train_rep_ans[indx]
    print(unique.shape,emb_api.shape,fam.shape)
    return unique,emb_api,fam,rep #改
train_emb,train_emb_api,train_fam_ans,train_rep_ans = unique_hash(train_emb,train_emb_api,train_fam_ans,train_rep_ans)

(3701, 213, 768) (3701, 213) (3701, 44)


In [20]:
train_rep_ans.shape

(3701, 213, 1)

In [5]:
def _shuffle(X, X2 ,X3,X4):
#     X3 = np.take(train_fam_ans,[0],axis=-1) #只train第幾個familiy
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize],X4[randomize])

In [6]:
train_emb, train_emb_api, train_fam_ans, train_rep_ans = _shuffle(train_emb, train_emb_api, train_fam_ans, train_rep_ans)
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = _shuffle(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans)

# test_emb, test_emb_api,test_fam_ans,test_rep_ans  = _shuffle(test_emb,test_emb_api,test_fam_ans,test_rep_ans)

print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)

# print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

train of sent2vec vector: (3701, 213, 768) (3701, 213) (3701, 44) (3701, 213, 1)
valid of sent2vec vector: (437, 213, 768) (437, 213) (437, 44) (437, 213, 1)


In [7]:
# scale = 'no'

# dim-wise scaling
def scaling(trainX,validX,testX,scale='min_max'):
#     if scale == 'min_max':
    max_value = max([np.max(trainX) , np.max(validX),np.max(testX)])
    min_value = min([np.min(trainX),np.min(validX),np.min(testX)])

    trainX = (trainX - min_value) / (max_value - min_value)
    validX = (validX - min_value) / (max_value - min_value )
    testX = (testX - min_value) / (max_value - min_value )
    print(np.max(trainX),np.max(validX))
    return trainX,validX,testX , max_value , min_value
def scaling(trainX,validX,testX,scale='mean_dim',):
#     if scale == 'min_max':
    alls = np.concatenate((trainX,validX,testX),axis=0)
    mean = np.mean(alls,axis=-1)
    mean = np.mean(mean,axis=0)
    mean = np.expand_dims(mean,axis=-1)
    mean = np.repeat(mean,trainX.shape[2],axis=-1)
    mean = np.expand_dims(mean,axis=0)
    mean_train = np.repeat(mean,trainX.shape[0],axis=0)
    mean_valid = np.repeat(mean,validX.shape[0],axis=0)
    mean_test = np.repeat(mean,testX.shape[0],axis=0)
    std = np.std(alls,axis=-1)
    std = np.std(std,axis=0)
    std = np.expand_dims(std,axis=-1)
    std = np.repeat(std,validX.shape[2],axis=-1)
    std_train = np.repeat(std,trainX.shape[0],axis=0)
    std_valid = np.repeat(std,validX.shape[0],axis=0)
    std_test = np.repeat(std,testX.shape[0],axis=0)
#     min_value = min([np.min(trainX),np.min(validX),np.min(testX)])

    trainX = (trainX - mean) / (std + 1e-10)
    validX = (validX - mean) / (std + 1e-10)
    testX = (testX - mean) / (std + 1e-10)
#     print(np.max(trainX),np.max(validX))
    return trainX,validX,testX , mean , std

# train_emb,valid_emb,test_emb , max_value,min_value = scaling(train_emb,valid_emb,test_emb)   
# print(valid_emb.shape)

In [8]:
sum(train_fam_ans)

array([2998, 4344, 1326,  867,  294,  234,  300,  141,  264,  272,  100,
        202,  202,   58,   76,  586,   74,   70,   97,   65,   76,  103,
        441,   69,   90,  114,   90,   44,   67,   58,  229,   42,   78,
         91,  151,  157,   41,   99,   20,   24,   38,   47,   30,  112])

In [9]:
train_emb.shape[2]

768

In [10]:
# kk = np.mean(train_emb,axis=-1)
# kk = np.mean(kk,axis=0)
# kk = np.expand_dims(kk,axis=0)
# kk = np.repeat(kk,100,axis=0)
# kk = np.expand_dims(kk,axis=-1)
# kk = np.repeat(kk,768,axis=-1)
# kk.shape

In [11]:
# kkk = (emb_matrix - kk)/kk


In [12]:
# kk = np.std(emb_matrix,axis=-1)
# kk = np.expand_dims(kk,axis=-1)
# kk = np.repeat(kk,768,axis=-1)
# kk

In [13]:
# bc = SMOTEENN()
# N,t,d = train_emb.shape
# train_emb_ = train_emb.reshape(N,t*d)
# train_fam_ans_ = train_fam_ans.reshape(N,)
# train_emb_ , train_fam_ans_  = bc.fit_resample(train_emb_, train_fam_ans_)

In [14]:
# train_emb = train_emb_.reshape(-1,t,d)
# train_fam_ans = train_fam_ans_.reshape(-1,1)

In [15]:
class_weights = sum(train_fam_ans) / sum(sum(train_fam_ans))
fam_weights={}
for i in range(len(class_weights)):
    fam_weights[i] = 1/class_weights[i]
fam_weights

all_fam = sum(train_fam_ans)
for i in range(len(all_fam)):
    fam_weights[i] = all_fam[i]
fam_weights

import math
def create_class_weight(labels_dict,mu=0.55): #0.79 #0.46 #改
    total = np.sum(np.array(list(labels_dict.values())))
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight
fam_weights = create_class_weight(fam_weights)
fam_weights

{0: 1.0043028310118869,
 1: 1.0,
 2: 1.8200813389286221,
 3: 2.244964532893888,
 4: 3.326423742335748,
 5: 3.5546823943167287,
 6: 3.306221035018229,
 7: 4.0612436192962615,
 8: 3.434054406528114,
 9: 3.4042014433784327,
 10: 4.404833323686339,
 11: 3.7017358122732253,
 12: 3.7017358122732253,
 13: 4.949560499128011,
 14: 4.679270169388099,
 15: 2.6366837200974174,
 16: 4.705938416470261,
 17: 4.761508267625071,
 18: 4.435292531171047,
 19: 4.835616239778793,
 20: 4.679270169388099,
 21: 4.375274521444794,
 22: 2.920958634227584,
 23: 4.77589700507717,
 24: 4.510193839344165,
 25: 4.273805061279934,
 26: 4.510193839344165,
 27: 5.2258138757561685,
 28: 4.805310890283464,
 29: 4.949560499128011,
 30: 3.5762815061201905,
 31: 5.272333891391061,
 32: 4.653294682984838,
 33: 4.49914400315758,
 34: 3.9927236728595057,
 35: 3.953757704326122,
 36: 5.296431442970122,
 37: 4.41488365953984,
 38: 6.014271236120439,
 39: 5.831949679326485,
 40: 5.372417349948044,
 41: 5.1598559079643715,
 42: 5.

In [16]:
# temp = list(fam_weights.values())
# max_value = np.max(temp)
# for i in range(len(fam_weights)):
#     fam_weights[i] = fam_weights[i]/max_value
# fam_weights

## model hyperparameters

In [7]:
emb_dim = 768 #被除數
num_heads = 48#除數，要整除
max_length = 213 # max sequence length
fam_num = train_fam_ans.shape[1]
vocabulary_size = 26
transformer_depth = 1
transformer_dropout = 0.1
l2_reg_penalty = 1e-6#1e-4
dp_rate = 0.15

traina = True #改
batch_size = 64 #改

In [8]:
constrain = keras.constraints.MinMaxNorm(min_value=0.0, max_value=0.0, rate=1.0, axis=0)
init = keras.initializers.Ones()
coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding')
act_layer = TransformerACT(
            name='adaptive_computation_time')

transformer_block = TransformerBlock(
            name='transformer', num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            # Allow bi-directional attention
            use_masking=False)
add_segment_layer = Add(name='add_segment')
l2_regularizer = (regularizers.l2(l2_reg_penalty) if l2_reg_penalty else None)

# Model

In [9]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
#shape=(max_length,emb_dim),,batch_shape=(batch_size,max_length,emb_dim)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
#shape=(max_length,),batch_shape=(batch_size,max_length)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length
                    ,trainable=False,name='api_emb')(sent_ids) #改



Instructions for updating:
Colocations handled automatically by placer.


In [10]:
segment_embeddings = Add()([sentemb,api_emb])
# next_step_input1 = RemoveMask()(segment_embeddings)
# next_step_input = coordinate_embedding_layer(next_step_input1, step=0,trainable=traina) #next_step_input_emb
# next_step_input= RestoreMask()([next_step_input,segment_embeddings])
# next_step_input = add_segment_layer([next_step_input, api_emb]) 

att_rnn = GRU(int(emb_dim/4),return_sequences=True,name='att_rnn',trainable=False)(segment_embeddings)
next_step_input = BatchNormalization(name='bn0')(att_rnn)
att_in = Dense(64,kernel_initializer=keras.initializers.lecun_normal(),activation='selu', # 改
               name='attention_in_64',trainable=True,kernel_regularizer=l2_regularizer)(next_step_input)
rep_prediction = (
        Dense(1, name='0_1_predict', activation='sigmoid') # hard_sigmoid
    (att_in))






In [11]:
mul = multiply([segment_embeddings,rep_prediction],name='mul')
# mul = BatchNormalization()(mul)
lstm,state_h,state_c = LSTM(int(emb_dim/4),return_sequences=True,return_state=True,name='family_in')(mul)
att_layer = MultiHeadAttention(
    head_num=48, trainable=True,
    name='Multi-Head')(lstm)


att_layer = BatchNormalization(name='bn1')(att_layer)
head_out = MultiHead(Dense(4,kernel_initializer=keras.initializers.lecun_normal(),activation='selu')
                     ,layer_num=2,name='Multi-Dense')(att_layer)
head_out1 = RemoveMask()(head_out)
flat = Flatten()(head_out1)
# flat = RestoreMask()([flat,att_layer])
con = Concatenate()([state_h,state_c])
flat = Concatenate()([con,flat])
bn = BatchNormalization(name='bn2')(flat)
dense2 = Dense(1024,kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
              name='dense2')(bn)
dp = Dropout(dp_rate)(dense2)
out = Dense(44,activation='sigmoid',name='family_out')(dp)
                     
# bn = BatchNormalization()
# dp = Dropout(dp_rate)
# dense1 = Dense(int(emb_dim/8),kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
#               kernel_regularizer=l2_regularizer,name='dense1')
# dense2 = Dense(32,kernel_initializer=keras.initializers.lecun_normal(),activation='selu',
#               name='dense2')
# dense3 = Dense(1,kernel_initializer=keras.initializers.lecun_normal(),activation='sigmoid',
#               name='dense3')
# gru = GRU(int(emb_dim/4), dropout=dp_rate, recurrent_dropout=dp_rate,name='gru_64')
# alls = []
# for i in range(fam_num):
#     alls.append(dense2(bn(gru(mul))))
# #     alls.append(dense3(dense2(dp(bn(dense1(BatchNormalization()(gru(mul))))))))
# out = Concatenate()(alls)
# out = BatchNormalization()(out)
# bn = BatchNormalization()
# all_out = []
# for i in range(fam_num):
#     all_out.append(dense3(bn(dense1(out))))
# out1 = Concatenate()(all_out)
# # out = Dense(44,activation='sigmoid',name='family_out')(out)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model = Model(inputs=[sent_ids1,sentemb1], outputs=[rep_prediction,out]) #out
model.load_weights('./model/att_clf/2ndStage_44fam_0613_copy1.h5')#,by_name=True) #改
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 213)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 213, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 213)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 213, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [13]:
model = multi_gpu_model(model , gpus=2)

# model.load_weights('./model/LSTM_att/1stStage_44fam_0607.h5')

In [24]:
'''

sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length,trainable=True,name='api_emb')(sent_ids)

final_emb = Add()([sentemb,api_emb])

timesteps,state_h,state_c = LSTM(int(emb_dim/2),return_sequences=True,return_state=True,name='lstm1')(final_emb) #final_emb
state = Concatenate()([state_h,state_c])
fc = Dense(max_length,activation='sigmoid',bias_constraint=None,kernel_initializer=init,name='attention')(state)
fc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='RasMMA')(fc)
fc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/2),axis=-1))(fc)
# fc = keras.backend.repeat_elements(fc,256,axis=-1)
# fc = keras.backend.expand_dims(fc,axis=-1)
mul = Multiply()([fc,timesteps])
# mul = BatchNormalization()(mul)
alls = []
gru = (GRU(int(emb_dim/4))) #/8
# gru = GRU(1)
bn = BatchNormalization()
dp = Dropout(0.01)

dense = Dense(1,activation='sigmoid')
for i in range(fam_num):
#     alls.append(dense(bn(gru(mul))))
    alls.append(dense(dp(bn(gru(mul)))))
#     alls.append(gru(mul))
out = Concatenate(name='family')(alls)
# out = Dense(44,activation='sigmoid')(out)
model_old = Model(inputs=[sent_ids1,sentemb1], outputs=[out]) #out
model_old = multi_gpu_model(model_old , gpus=3)
model_old.load_weights('./model/LSTM_att/1stStage_44fam_0607.h5')
model_old.summary()

# model = load_model('./model/LSTM_att/1stStage_44fam_0607.h5_all.h5')'''

"\n\nsentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')\nsentemb = Masking(mask_value=0)(sentemb1)\nsent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID\nsent_ids = Masking(mask_value=0)(sent_ids1)\napi_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length,trainable=True,name='api_emb')(sent_ids)\n\nfinal_emb = Add()([sentemb,api_emb])\n\ntimesteps,state_h,state_c = LSTM(int(emb_dim/2),return_sequences=True,return_state=True,name='lstm1')(final_emb) #final_emb\nstate = Concatenate()([state_h,state_c])\nfc = Dense(max_length,activation='sigmoid',bias_constraint=None,kernel_initializer=init,name='attention')(state)\nfc = Lambda(lambda x: keras.backend.expand_dims(x,axis=-1),name='RasMMA')(fc)\nfc = Lambda(lambda x: keras.backend.repeat_elements(x,int(emb_dim/2),axis=-1))(fc)\n# fc = keras.backend.repeat_elements(fc,256,axis=-1)\n# fc = keras.backend.expand_dims(fc,axis=-1)\nmul = Multiply()([fc,timest

In [25]:
# model_old.layers[-2].save_weights('./model/LSTM_att/test.h5')

In [14]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
from keras.metrics import top_k_categorical_accuracy
def custom_acc1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred,k=3)
from keras.metrics import binary_accuracy
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

def precision(y_true, y_pred):
    return tf.keras.metrics.Precision(y_true,y_pred)[1]
def recall(y_true, y_pred):
    return tf.keras.metrics.Recall(y_true,y_pred)[1]
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))
def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))
# from sklearn.metrics import f1_score
# def f1_sk(y_true,y_pred):
#     score = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
#     return score

# 訓練參數
los = [losses.binary_crossentropy,binary_focal_loss(alpha=.25, gamma=2)] # 1st stage.  f1_loss
#SINGLE
# los = [binary_focal_loss(alpha=.25, gamma=2)] #改
# los = [hn_multilabel_loss]
# los = [losses.binary_crossentropy]
# MML
'''los = []
for i in range(fam_num):
    los.append(binary_focal_loss(alpha=.25, gamma=2))
los = [losses.binary_crossentropy] + los'''


metric = {'0_1_predict': bin_acc,'family_out': f1_metric} # 1st stage. km.f1_score()
#SINGLE
# metric = [f1_metric,bin_acc]
# metric = [km.f1_score(),bin_acc,km.binary_f1_score()]
# metric = {'RasMMA': 'acc'}
# metric = [bin_acc] #改
#MML
'''metrics = []
for i in range(fam_num+1):
    metrics.append('acc')
# metrics = {}
# metrics['RasMMA'] = 'acc'
# for i in range(fam_num):
#     metrics['fam'+str(i)]='acc'
metric = metrics'''


loss_weight = [1,1] #stage1 0.95,0.05  #1st stage # 2nd stage [0.01,0.99]
#SINGLE
# loss_weight = [1]
#MML
'''loss_weight = []
for i in range(fam_num):
    loss_weight.append(0.95)
loss_weight = [0.05] + loss_weight'''

learning_rate = 5e-4#2e-4 # 2nd stage: 1e-4 @1st:2e-4 0.002
# batch_size = 128 #32 #128

num_epochs = 1000
patien = 50

model_save_path = './model/att_clf/3rdStage_44fam_0613_copy1.h5' #改
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0]

In [15]:
# import tensorflow as tf
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False) #clipnorm=1. , clipvalue=1.
optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.) #改
# tf.keras.optimizers.Nadam
lr_scheduler1 = callbacks.LearningRateScheduler(
        CosineLRSchedule(lr_high=0.0005, lr_low=1e-8, #learning_rate #改
                         initial_period=num_epochs),
        verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/3),
                                      min_lr=1e-8,mode='min')

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)#{'word_predictions': masked_perplexity})

In [16]:
# save best, early stopping, 2 models ens weight:(best=0.8,last=0.2)
history = History()
stop_nan = callbacks.TerminateOnNaN()
model_callbacks = [
        callbacks.ModelCheckpoint(
            model_save_path, #val_f1_metric,max。val_family_out_f1_metric
            monitor='val_loss',mode='min' ,save_best_only=True, verbose=1,save_weights_only=True), #改
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
        lr_scheduler, lr_scheduler1,history,stop_nan
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

In [21]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = class_prediction: shape為(N, max_seq_length, 1)
    Y_train2 = family_prediction(stage2): shape為(N, fam_num)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
            train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
#             yield(train_X2,train_Y2)
            yield ([train_X1,train_X2],[train_Y1,train_Y2]) #ori
#             yield ([train_X1,train_X2],[train_Y2]) #改
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break
            
#     data_size = X_train.shape[0]
#     ep = data_size / batch_size

In [22]:

H = model.fit_generator(
    generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_rep_ans,Y_train2=train_fam_ans) #Y_train2
#                     generator=training_batch(batch_size=batch_size,X_train1=valid_emb_api,X_train2=valid_emb ,
#                                              Y_train1=train_rep_ans,Y_train2=train_fam_ans)
                        , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans,valid_fam_ans]) #ori
#                    ,validation_data= (valid_emb, valid_fam_ans) 
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans,valid_fam_ans]) #ori #改
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2) #validY2
                    ,max_queue_size=10  #,class_weight=fam_weights #改
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")
#1st:train 0_1_prediction=0.14XX

Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.0005.
58/58 [==============================] - 39s 670ms/step - loss: 16.9456 - 0_1_predict_loss: 0.7345 - family_out_loss: 16.2110 - 0_1_predict_bin_acc: 0.5401 - family_out_f1_metric: 0.3684 - val_loss: 52.2827 - val_0_1_predict_loss: 0.7243 - val_family_out_loss: 51.5582 - val_0_1_predict_bin_acc: 0.4357 - val_family_out_f1_metric: 0.3915

Epoch 00001: val_loss improved from inf to 52.28265, saving model to ./model/att_clf/3rdStage_44fam_0613_copy1.h5
Epoch 2/1000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0004999987663251386.
58/58 [==============================] - 38s 650ms/step - loss: 15.1826 - 0_1_predict_loss: 0.6901 - family_out_loss: 14.4924 - 0_1_predict_bin_acc: 0.5823 - family_out_f1_metric: 0.4077 - val_loss: 12071.8386 - val_0_1_predict_loss: 0.7652 - val_family_out_loss: 12071.0734 - val_0_1_predict_bin_acc: 0.4342 - val_family_out_f1_metric: 0.0680

Epoch 00002: val_loss d

58/58 [==============================] - 36s 616ms/step - loss: 14.4237 - 0_1_predict_loss: 0.6611 - family_out_loss: 13.7625 - 0_1_predict_bin_acc: 0.5977 - family_out_f1_metric: 0.4818 - val_loss: 79.8159 - val_0_1_predict_loss: 0.7054 - val_family_out_loss: 79.1104 - val_0_1_predict_bin_acc: 0.4371 - val_family_out_f1_metric: 0.4427

Epoch 00017: val_loss did not improve from 42.05258
Epoch 18/1000

Epoch 00018: LearningRateScheduler setting learning rate to 0.0004996435524089718.
58/58 [==============================] - 36s 617ms/step - loss: 15.6650 - 0_1_predict_loss: 0.6678 - family_out_loss: 14.9971 - 0_1_predict_bin_acc: 0.5667 - family_out_f1_metric: 0.4615 - val_loss: 127.1858 - val_0_1_predict_loss: 0.6963 - val_family_out_loss: 126.4894 - val_0_1_predict_bin_acc: 0.4347 - val_family_out_f1_metric: 0.2823

Epoch 00018: val_loss did not improve from 42.05258
Epoch 19/1000

Epoch 00019: LearningRateScheduler setting learning rate to 0.0004996003955194932.
58/58 [=============


Epoch 00033: val_loss did not improve from 41.58961
Epoch 34/1000

Epoch 00034: LearningRateScheduler setting learning rate to 0.0004986577298437981.
58/58 [==============================] - 36s 619ms/step - loss: 14.4697 - 0_1_predict_loss: 0.6539 - family_out_loss: 13.8157 - 0_1_predict_bin_acc: 0.6218 - family_out_f1_metric: 0.4890 - val_loss: 69.5720 - val_0_1_predict_loss: 0.7654 - val_family_out_loss: 68.8065 - val_0_1_predict_bin_acc: 0.4358 - val_family_out_f1_metric: 0.4199

Epoch 00034: val_loss did not improve from 41.58961
Epoch 35/1000

Epoch 00035: LearningRateScheduler setting learning rate to 0.0004985752260952978.
58/58 [==============================] - 36s 621ms/step - loss: 14.2476 - 0_1_predict_loss: 0.6470 - family_out_loss: 13.6005 - 0_1_predict_bin_acc: 0.6405 - family_out_f1_metric: 0.5154 - val_loss: 70.6764 - val_0_1_predict_loss: 0.7723 - val_family_out_loss: 69.9040 - val_0_1_predict_bin_acc: 0.4366 - val_family_out_f1_metric: 0.3983

Epoch 00035: val_loss

58/58 [==============================] - 36s 618ms/step - loss: 14.6944 - 0_1_predict_loss: 0.6746 - family_out_loss: 14.0197 - 0_1_predict_bin_acc: 0.5599 - family_out_f1_metric: 0.4889 - val_loss: 2231.9745 - val_0_1_predict_loss: 0.6924 - val_family_out_loss: 2231.2822 - val_0_1_predict_bin_acc: 0.5628 - val_family_out_f1_metric: 0.0945

Epoch 00050: val_loss did not improve from 39.98013
Epoch 51/1000

Epoch 00051: LearningRateScheduler setting learning rate to 0.0004969221467070815.
58/58 [==============================] - 36s 621ms/step - loss: 15.2637 - 0_1_predict_loss: 0.6818 - family_out_loss: 14.5818 - 0_1_predict_bin_acc: 0.5792 - family_out_f1_metric: 0.4990 - val_loss: 553.1449 - val_0_1_predict_loss: 0.6776 - val_family_out_loss: 552.4672 - val_0_1_predict_bin_acc: 0.5627 - val_family_out_f1_metric: 0.1333

Epoch 00051: val_loss did not improve from 39.98013
Epoch 52/1000

Epoch 00052: LearningRateScheduler setting learning rate to 0.0004967980675386349.
58/58 [=========

58/58 [==============================] - 36s 619ms/step - loss: 16.4820 - 0_1_predict_loss: 0.6686 - family_out_loss: 15.8133 - 0_1_predict_bin_acc: 0.5993 - family_out_f1_metric: 0.4318 - val_loss: 46.4079 - val_0_1_predict_loss: 0.6956 - val_family_out_loss: 45.7122 - val_0_1_predict_bin_acc: 0.4511 - val_family_out_f1_metric: 0.4400

Epoch 00067: val_loss did not improve from 39.98013
Epoch 68/1000

Epoch 00068: LearningRateScheduler setting learning rate to 0.0004944824453109813.
58/58 [==============================] - 36s 622ms/step - loss: 15.1099 - 0_1_predict_loss: 0.6576 - family_out_loss: 14.4522 - 0_1_predict_bin_acc: 0.6189 - family_out_f1_metric: 0.4657 - val_loss: 490.2940 - val_0_1_predict_loss: 0.8227 - val_family_out_loss: 489.4712 - val_0_1_predict_bin_acc: 0.4423 - val_family_out_f1_metric: 0.1916

Epoch 00068: val_loss did not improve from 39.98013
Epoch 69/1000

Epoch 00069: LearningRateScheduler setting learning rate to 0.0004943171445514306.
58/58 [=============

58/58 [==============================] - 36s 620ms/step - loss: 16.0094 - 0_1_predict_loss: 0.6661 - family_out_loss: 15.3432 - 0_1_predict_bin_acc: 0.5863 - family_out_f1_metric: 0.4671 - val_loss: 76.5065 - val_0_1_predict_loss: 0.7849 - val_family_out_loss: 75.7215 - val_0_1_predict_bin_acc: 0.4488 - val_family_out_f1_metric: 0.3506

Epoch 00084: val_loss did not improve from 39.98013
Epoch 85/1000

Epoch 00085: LearningRateScheduler setting learning rate to 0.0004913455828001244.
58/58 [==============================] - 36s 623ms/step - loss: 15.3352 - 0_1_predict_loss: 0.6735 - family_out_loss: 14.6616 - 0_1_predict_bin_acc: 0.5942 - family_out_f1_metric: 0.4709 - val_loss: 59.6483 - val_0_1_predict_loss: 0.7211 - val_family_out_loss: 58.9271 - val_0_1_predict_bin_acc: 0.4480 - val_family_out_f1_metric: 0.3789

Epoch 00085: val_loss did not improve from 39.98013
Epoch 86/1000

Epoch 00086: LearningRateScheduler setting learning rate to 0.0004911395318273572.
58/58 [===============

# Evaluate
* multi model weights 儲存

In [25]:
model_ = model.layers[-3]
# model_.load_weights(model_save_path)
model_.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 213)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 213, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 213)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 213, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [26]:
model.load_weights(model_save_path)
model_ = model.layers[-3]
model_.save_weights(model_save_path)
model_.summary()
pickle.dump(file=open(tensorboard_log_path + '/'+'2mdStage_H.pkl','wb'),obj=H) #改


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 213)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 213, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 213)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 213, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [27]:
from keras.models import model_from_json, model_from_yaml
json_string = model_.to_json()
yaml_string = model_.to_yaml()
pickle.dump(file=open(tensorboard_log_path + '/'+'3rdStage_arch.pkl','wb'),obj=(json_string,yaml_string))

In [28]:
# model.load_weights('./model/LSTM_att/1stStage_44fam_0610.h5')
# model.load_weights('./model/att_clf/1stStage_44fam_0611_copy1.h5')
# score = model.evaluate([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2)
print(len(test_emb_api)) #改
ans = model_.predict([test_emb_api,test_emb]) #改
y_true = test_fam_ans #改
# ans = model.predict([valid_emb_api,valid_emb])
len(ans)

424


2

In [34]:
test_rep_ans[113]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [32]:
ans[1][14]

array([1.36896938e-01, 1.77779198e-02, 7.04928935e-02, 1.18593872e-01,
       2.69446373e-02, 7.34211206e-02, 1.49410665e-02, 7.76961446e-03,
       4.66791689e-01, 3.26322019e-02, 1.88965291e-01, 1.51605010e-02,
       1.64790809e-01, 5.48806787e-03, 3.32233906e-02, 1.64065361e-02,
       7.89082050e-03, 9.24912095e-03, 2.24945843e-02, 1.36159956e-02,
       2.29618847e-02, 3.71900201e-03, 1.18773758e-01, 4.05153632e-03,
       1.84470415e-03, 1.03324264e-01, 4.15176153e-04, 6.51049614e-03,
       3.95926833e-03, 1.31236911e-02, 2.74758041e-02, 2.02247500e-03,
       6.20424747e-04, 1.45375729e-03, 2.78486609e-02, 5.04574180e-03,
       1.30167603e-03, 9.90450382e-04, 9.85860825e-05, 5.89907169e-04,
       5.48958778e-04, 1.13338232e-03, 1.50439143e-03, 4.75129485e-03],
      dtype=float32)

In [36]:
# model_.save_weights('./model/LSTM_att/test4.h5')

In [34]:
# model_ = model.layers[-2]
layer_name = 'lambda_1' #lambda_1 multiply_1  #9~12
intermediate_layer_model = Model(inputs=model_.inputs,
                                 outputs=model_.layers[9].output)
intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
intermediate_output[113]

array([[0.99636805],
       [0.76627016],
       [0.814659  ],
       [0.91933   ],
       [0.87644374],
       [0.8787677 ],
       [0.8807063 ],
       [0.88174474],
       [0.8824611 ],
       [0.8829738 ],
       [0.8834057 ],
       [0.8836584 ],
       [0.883814  ],
       [0.8839257 ],
       [0.8840245 ],
       [0.884064  ],
       [0.88410145],
       [0.8841282 ],
       [0.88415104],
       [0.8841632 ],
       [0.8841717 ],
       [0.8841779 ],
       [0.88418305],
       [0.8841859 ],
       [0.8841878 ],
       [0.8841892 ],
       [0.8841904 ],
       [0.88419104],
       [0.8841915 ],
       [0.88419175],
       [0.8841921 ],
       [0.8841922 ],
       [0.88419235],
       [0.88419235],
       [0.88419247],
       [0.8841925 ],
       [0.8841925 ],
       [0.8841925 ],
       [0.8841925 ],
       [0.88419265],
       [0.8841925 ],
       [0.8841925 ],
       [0.8841925 ],
       [0.8841926 ],
       [0.8841925 ],
       [0.92568326],
       [0.92568755],
       [0.884

In [38]:
(sum(intermediate_output))

array([226.29732], dtype=float32)

In [35]:
intermediate_output[113].shape

(213, 1)

In [48]:
intermediate_output[190]

array([[0.99933076],
       [0.7101693 ],
       [0.8042407 ],
       [0.844324  ],
       [0.8649031 ],
       [0.86269283],
       [0.9070231 ],
       [0.90702283],
       [0.9070228 ],
       [0.9070234 ],
       [0.9070228 ],
       [0.90702295],
       [0.9070232 ],
       [0.9070231 ],
       [0.90702295],
       [0.907023  ],
       [0.90702343],
       [0.9070246 ],
       [0.90702295],
       [0.90702295],
       [0.90702236],
       [0.8869482 ],
       [0.8869499 ],
       [0.88696206],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886961  ],
       [0.886

In [41]:
model_.summary() #multiply_1 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 213)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 213, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 213)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 213, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [42]:
sum(intermediate_output)

array([226.29732], dtype=float32)

In [51]:
from sklearn.metrics import f1_score,recall_score,precision_score
# y_true = np.squeeze(test_fam_ans)
# y_true = np.squeeze(valid_fam_ans1)
# y_pred = np.squeeze(predict_fam)
final_ans = []
for sample in ans[1]:
    sample_ans = []
    for value in sample:
        if value < 0.33: #0.26 #0.33
            sample_ans.append(0)
        else:
            sample_ans.append(1)
    final_ans.append(sample_ans)
final_ans = np.array(final_ans)
print(final_ans.shape , sum(final_ans[0]))

(424, 44) 2


In [52]:
# y_true = test_fam_ans
print(y_true.shape , final_ans.shape)
recall = recall_score(y_true=y_true, y_pred=final_ans, average='micro')
precision = precision_score(y_true=y_true, y_pred=final_ans, average='micro')
f1 = f1_score(y_true=y_true, y_pred=final_ans, average='micro')
recall ,precision, f1

(424, 44) (424, 44)


(0.4930232558139535, 0.48036253776435045, 0.486610558530987)